# GDP Data Extraction and Processing of the Top 10 Global Economies

The goal of this project is to build an automated Python-based data pipeline that:
- Extracts real-world GDP data from a live (archived) Wikipedia source using web scraping.
- Processes and cleans the data using pandas and numpy to isolate the top 10 largest global economies, ranked by nominal GDP (as estimated by the IMF).
- Converts and standardizes the values from million USD to billion USD, rounded to two decimal places for consistency.
- Exports the final data into a structured CSV file (Largest_economies.csv) for future use in analytics or business intelligence.

This script is part of a broader effort to support data-driven market entry analysis for a multinational firm seeking global expansion.

## 1. Web Scraping using Numpy & Pandas

In [32]:
# Import Libraries
import numpy as np 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')  # Suppress warnings

- Extract the required GDP data from the given URL using Web Scraping.

In [33]:
# Load the archived Wikipedia page
URL = "https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29"
tables = pd.read_html(URL)

- Select Target Table which contains nominal GDP data by country.
- Check the table structure using .shape and .head() to confirm column indexes.

In [34]:
# Preview structure of the 4th table (index 3)
df = tables[3]
print("Columns:", df.columns)
print(df.head())

Columns: MultiIndex([( 'Country/Territory', 'Country/Territory'),
            (         'UN region',         'UN region'),
            (        'IMF[1][13]',          'Estimate'),
            (        'IMF[1][13]',              'Year'),
            (    'World Bank[14]',          'Estimate'),
            (    'World Bank[14]',              'Year'),
            ('United Nations[15]',          'Estimate'),
            ('United Nations[15]',              'Year')],
           )
  Country/Territory UN region IMF[1][13]            World Bank[14]             \
  Country/Territory UN region   Estimate       Year       Estimate       Year   
0             World         —  105568776       2023      100562011       2022   
1     United States  Americas   26854599       2023       25462700       2022   
2             China      Asia   19373586  [n 1]2023       17963171  [n 3]2022   
3             Japan      Asia    4409738       2023        4231141       2022   
4           Germany    Europe    43

In [35]:
# Replace the column headers with column numbers
df.columns = range(df.shape[1])
print("Shape:", df.shape)  # Check column count

Shape: (214, 8)


- Keep only the country name and IMF GDP value columns 
- Retain rows 1 to 10 to get the top 10 largest economies.

In [36]:
# Retain columns with index 0 and 2 (name of country and value of GDP quoted by IMF)
df = df[[0,2]]

# Retain the Rows with index 1 to 10, indicating the top 10 economies of the world.
df = df.iloc[1:11,:]

# Assign column names as "Country" and "GDP (Million USD)"
df.columns = ['Country', 'GDP (Million USD)']

# Display result
df

,Country,GDP (Million USD)
1,United States,26854599
2,China,19373586
3,Japan,4409738
4,Germany,4308854
5,India,3736882
6,United Kingdom,3158938
7,France,2923489
8,Italy,2169745
9,Canada,2089672
10,Brazil,2081235


- Change GDP values to integers to allow numerical operations.
- Convert to Billions: Divide GDP by 1000 to convert from million to billion USD.

In [37]:
# Change the data type of the 'GDP (Million USD)' column to integer
df['GDP (Million USD)'] = df['GDP (Million USD)'].astype(int)

# Convert the GDP value from Million USD to Billion USD
df[['GDP (Million USD)']] = df[['GDP (Million USD)']]/1000

# Round the value to 2 decimal places
df[['GDP (Million USD)']] = np.round(df[['GDP (Million USD)']], 2)

# Rename the column header from 'GDP (Million USD)' to 'GDP (Billion USD)'
df. rename (columns = {'GDP (Million USD)' : 'GDP (Billion USD)'})

,Country,GDP (Billion USD)
1,United States,26854.60
2,China,19373.59
3,Japan,4409.74
4,Germany,4308.85
5,India,3736.88
6,United Kingdom,3158.94
7,France,2923.49
8,Italy,2169.74
9,Canada,2089.67
10,Brazil,2081.24


- Save the final DataFrame to a CSV file named Largest_economies.csv.

In [38]:
# Load the DataFrame to the CSV file named "Largest_economies.csv"
df.to_csv('Largest_economies.csv', index=False)

## 2. Webscraping using Requests, BeautifulSoup & Pandas

This task demonstrates how to extract and clean structured data from a web page (Wikipedia GDP table) using Python’s web scraping tools — specifically requests, BeautifulSoup, and pandas.

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 

In [42]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 

In [43]:
'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'

'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'

In [44]:
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_attribs = ["Country", "GDP_USD_millions"]
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = '/Users/zakariyaboutayeb/Documents/12. IBM /02. Data Engineering/Countries_by_GDP.csv'

### Extracting information



In [45]:
def extract(url, table_attribs):
    page = requests.get(url).text
    data = BeautifulSoup(page,'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[2].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col)!=0:
            if col[0].find('a') is not None and '—' not in col[2]:
                data_dict = {"Country": col[0].a.contents[0],
                             "GDP_USD_millions": col[2].contents[0]}
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df,df1], ignore_index=True)
    return df

In [46]:
df = extract(url, table_attribs)
df.head()

,Country,GDP_USD_millions
0,United States,"26,854,599"
1,China,"19,373,586"
2,Japan,"4,409,738"
3,Germany,"4,308,854"
4,India,"3,736,882"


## Transform information

This task focuses on cleaning and transforming the GDP values extracted in Task 1 to prepare them for analysis.

- Convert the contents of the 'GDP_USD_millions' column of df
dataframe from currency format to floating numbers.
- Divide all these values by 1000 and round it to 2 decimal places.
- Modify the name of the column from 'GDP_USD_millions' to
'GDP_USD_billions'.

In [47]:
def transform(df):
    # Convert millions to billions and round to 2 decimal places
    GDP_list = df["GDP (Million USD)"].tolist()
    GDP_list = [np.round(x / 1000, 2) for x in GDP_list]

    # Replace column and rename it
    df["GDP (Million USD)"] = GDP_list
    df = df.rename(columns={"GDP (Million USD)": "GDP_USD_billions"})

    return df

In [48]:
print(df.columns.tolist())

['Country', 'GDP_USD_millions']


In [49]:
def transform(df):
    # Auto-detect column with 'GDP'
    gdp_col = [col for col in df.columns if 'GDP' in col][0]

    # Just round (no division)
    df[gdp_col] = np.round(df[gdp_col], 2)

    # Rename column
    df = df.rename(columns={gdp_col: "GDP_USD_billions"})
    
    return df

df = transform(df)
df

### Loading Information


Save the pandas DataFrame to a CSV (Comma-Separated Values) file. A file named Countries_by_GDP.csv is already created as well as a database called db_name = 'World_Economies.db'

In [55]:
def load_to_csv(df, csv_path):
    df.to_csv(csv_path)

Next step is to save the dataFrame (df) to a table in a SQL database in order it can be queried using SQL.

In [56]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

### Querying the database table

Let's run a SQL query on a database and display the results.We need to create a database connection first using SQLite.

In [57]:
def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

In [58]:
import sqlite3

# Connect to database 
conn = sqlite3.connect("World_Economies.db")

# Load DataFrame into database as table
df.to_sql("Countries_by_GDP", conn, if_exists='replace', index=False)

# Run the query
query = "SELECT * FROM Countries_by_GDP LIMIT 5;"
run_query(query, conn)

# Optional: Close connection
conn.close()

SELECT * FROM Countries_by_GDP LIMIT 5;
         Country GDP_USD_millions
0  United States       26,854,599
1          China       19,373,586
2          Japan        4,409,738
3        Germany        4,308,854
4          India        3,736,882


### Logging Progress

Logs a message with a timestamp to a text file called etl_project_log.txt. It’s used in ETL (Extract, Transform, Load) pipelines to track the progress of different steps.

In [62]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("/Users/zakariyaboutayeb/Documents/12. IBM /02. Data Engineering/etl_project_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

In [63]:
log_progress("Loaded data into SQL database")
log_progress("Extracted GDP data from Wikipedia")

### Function Calls

Let's outlines a complete ETL (Extract, Transform, Load) process with logging. Each line of code plays a specific role in tracking, transforming, saving, and querying data.

In [65]:
# Logs the start of the ETL pipeline to a text file for monitoring/debugging
log_progress('Preliminaries complete. Initiating ETL process')

# Extracts data from a webpage using the extract() function
df = extract(url, table_attribs)

# Logs the completion of extraction and start of transformation
log_progress('Data extraction complete. Initiating Transformation process')

# Transforms the data and loading it
df = transform(df)
log_progress('Data transformation complete. Initiating loading process')

# Save the transformed data to a CSV file on disk and log it
load_to_csv(df, csv_path)
log_progress('Data saved to CSV file')

# Creates SQLite database called World_Economies.db and log it
sql_connection = sqlite3.connect('World_Economies.db')
log_progress('SQL Connection initiated.')

# Load the df DataFrame into a SQL table named table_name and log it
load_to_db(df, sql_connection, table_name)
log_progress('Data loaded to Database as table. Running the query')

# Define and execute a SQL filter query
query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
run_query(query_statement, sql_connection)

log_progress('Process Complete.')

sql_connection.close()

SELECT * from Countries_by_GDP WHERE GDP_USD_billions >= 100
              Country GDP_USD_billions
0       United States       26,854,599
1               China       19,373,586
2               Japan        4,409,738
3             Germany        4,308,854
4               India        3,736,882
..                ...              ...
166  Marshall Islands              291
167             Palau              262
168          Kiribati              248
169             Nauru              151
170            Tuvalu               65

[171 rows x 2 columns]
